<a href="https://colab.research.google.com/github/gburgon/ukp-continuos-reformulation/blob/main/modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomo
!apt-get update -qq
!pip install pyomo
!pip install cyipopt
!pip install highspy pyomo
!pip install -q condacolab
condacolab.install()
!conda install pyscipopt
!apt-get install -y coinor-highs
import condacolab
import numpy as np
import pyomo.environ as pyo
import pandas as pd

# Hiperparameters
#(k=1-> binário)
k = 1
alpha = 0 # Penalização

# Creates the pyomo model
model = pyo.ConcreteModel()

# Bounds of i and j indices
model.I = pyo.RangeSet(0,n-1)
model.J = pyo.RangeSet(0,k-1) if k>0 else pyo.RangeSet(0,0)

# Defines Xij in [0,2^k]
if k>1:
  model.x = pyo.Var(model.I, model.J, bounds=(0,2**(k)))
else:
  model.x = pyo.Var(model.I,model.J, bounds=(0,1))
#Obs: Xi belongs to [0,2^(k+1)-1] because its a sum of the variables Xij

#Objective Function
def func_obj(m):
    return sum(v[i]*m.x[i,j] for i in m.I for j in m.J) + alpha*sum(m.x[i,j]*(m.x[i,j]-2**j) for i in m.I for j in m.J) #"Linear" penalization
    #return sum(v[i]*m.x[i,j] for i in m.I for j in m.J) - alpha*sum((m.x[i,j]*(m.x[i,j]-2**j))**2 for i in m.I for j in m.J) #Quadratic penalization
model.obj = pyo.Objective(rule=func_obj, sense=pyo.maximize)

#Capacity Constrain
def restricao_peso(m):
    return sum(p[i]*m.x[i,j] for i in m.I for j in m.J) <= L
model.cap = pyo.Constraint(rule=restricao_peso)

#Quadratic Constrains
def restricoes_quadraticas(m):
    return sum(m.x[i,j]*(m.x[i,j]-2**j) for i in m.I for j in m.J) == 0
model.quad_global = pyo.Constraint(rule=restricoes_quadraticas)

#Solves and print solution

solver = pyo.SolverFactory("scip")
solver.options["limits/gap"] = 0.1 # Minimum gap between primal/dual (Stopping criterion)!!
result = solver.solve(model, tee=True)

#Imprime resultados
print("Status:", result.solver.status)
print("Objetivo máximo:", pyo.value(model.obj))

